In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.interpolate import splrep, splev
import datetime as dt
import matplotlib.pyplot as plt
from fbprophet import Prophet
from scipy.interpolate import InterpolatedUnivariateSpline
%matplotlib inline
plt.rcParams.update({'font.size': 18})

In [2]:
def smape(satellite_predicted_values, satellite_true_values):
    # the division, addition and subtraction are pointwise
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/
                (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))


def drop_close(t, x, eps=10**9):
    '''
    t = time array, x = data array, eps is in nanoseconds
    Returns entries in t,x with corresponding consecutive times > eps
    
    '''
    t = np.array(t) #if not already np array, convert
    x = np.array(x) #if t or x are pandas Series, will have dimension mismatch
    far = np.concatenate([(t[1:] - t[:-1]) > eps, [True]])
    return t[far], x[far]


def resample(t, x, step=10 * 10**9, t_new=None, return_t=False):
    '''
    t: time array (or series); 
    x: data array (or series); 
    t_new: new time scale from start to end of t with step size step;
    step: = 10 seconds by default; 
    return_t: by default, do not return resampled times
    
    resample time series or array by 10 (default) sec intervals and 
    return new time series (if t_new=True) and spline approximation series for data
    
    
    '''
    t, x = drop_close(t, x)
    if t_new is None:
        t_new = np.arange(t[0], t[-1], step)
    try:
        spl = splrep(t, x)
        x_new = splev(t_new, spl)
    except:
        raise ValueError(f'interpolation error, x length = {len(x)}, \
        t_new length = {len(t_new)}')

    return (t_new, x_new) if return_t else x_new


def get_peaks(array):
    '''
    returns index of "sharp" peaks, excluding first and last values of array
    
    index of "smooth peaks", e.g. 1 2 3 9 9 3 2 1, is not returned
    '''
    return np.where((array[1:-1] > array[2:]) & (array[1:-1] > array[:-2]))[0] + 1


def get_satellite_data(data, sat_id):
    '''
    returns all data for particular satellite by id
    '''
    return data[data['sat_id'] == sat_id]


def remove_time_jumps(satellite_data, features_list=('x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'),
                      suffix='_jumps_removed',time_threshold = 0.00003):
    #time_threshold 0.00003 sufficient for test and train
    #time_threshold 0.00002 will throw errors
    '''
    removes time jumps in the simulation for a single satellite
    for train and test data, sufficient to set time_threshold at default
    '''
    satellite_data['t'] = ((pd.to_datetime(satellite_data['epoch']) - pd.to_datetime(satellite_data['epoch'])[0]) /
                           np.timedelta64(1, 'D')).astype(float)
    satellite_data['dt'] = satellite_data['t'].diff()
    # the most frequent time interval
    t_standard = satellite_data['dt'].value_counts().index[0]

    # time steps used for simulations
    satellite_data['t_sim'] = satellite_data.index.values * t_standard

    n = satellite_data.shape[0]
    corrected_features = []
    for feature_name in features_list:
        corrected_feature = [0] * n
        corrected_feature[0] = satellite_data[feature_name][0]

        feature = satellite_data[feature_name]

        for j in range(1, n - 1):
            if satellite_data.t[j] < satellite_data.t_sim[j] - time_threshold:
                # approximate by the left side
    
                # look for the interval
                step = 0
                while satellite_data.t[j] < satellite_data.t_sim[j - step] - time_threshold:
                    step += 1
                #             print(step)
                corrected_feature[j] = feature[j - step] - (satellite_data.t_sim[j - step] - satellite_data.t[j]) / (
                            satellite_data.t_sim[j - step] - satellite_data.t_sim[j - step - 1]) * (
                                              feature[j - step] - feature[j - step - 1])
            elif satellite_data.t[j] > satellite_data.t_sim[j] + time_threshold:
                # approximate by the right side
    
                # look for the interval
                step = 0
                while satellite_data.t[j] > satellite_data.t_sim[j + step] + time_threshold:
                    step += 1
    
                corrected_feature[j] = feature[j + step + 1] - (satellite_data.t_sim[j + step + 1] - satellite_data.t[j]) / (
                            satellite_data.t_sim[j + step + 1] - satellite_data.t_sim[j + step]) * (
                                              feature[j + step + 1] - feature[j + step])
            else:
                corrected_feature[j] = feature[j]
    
        corrected_feature[n - 1] = feature[n - 1] + corrected_feature[n - 2] - feature[n - 2]
        corrected_features.append(corrected_feature)
    return pd.DataFrame(corrected_features, index=[f + suffix for f in features_list]).T

In [3]:
def insert_previous_and_shift(df,col_name,ind):
    '''
    input a data frame (df), column name (col_name), and index (ind)
    insert previous value of df[col_name] at index and shift the rest 
    of df[col_name] from ind by +1;
    This is used for remove_time_jumps_fast
    '''
    shifted_series = df[col_name].shift(1)
    df[col_name].iloc[ind] = df[col_name].iloc[ind-1]
    df[col_name].iloc[ind+1:] = shifted_series.iloc[ind+1:]
    return df

In [4]:
def remove_time_jumps_fast(data, features_list=
                           ('x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'),
                           threshold = 0.000001):
    #time_threshold 0.00003 sufficient for test and train
    #time_threshold 0.00002 will throw errors
    '''
    removes time jumps in the simulation for a single satellite
    for train and test data, sufficient to set time_threshold at default
    s_data = satellite data
    the features are replaced by the correction
    note that threshold here is not the same as in remove_time_jumps
    '''
    data['t'] = ((pd.to_datetime(data['epoch']) - pd.to_datetime(data['epoch'])[0]) /
                               np.timedelta64(1, 'D')).astype(float)
    data['dt'] = data['t'].diff(1)

    index_for_correction = data[data['dt'] < threshold].index 
    #print(index_for_correction)
    if list(index_for_correction): #if non empty
        for feature in features_list:
            for i in index_for_correction:
                data = insert_previous_and_shift(data,feature,i)
    return data

## Transforming all the data

In [5]:
import utils
from LinearAlignment import LinearAlignment

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern 'utils_edit_mb.ipynb' matched no files


In [6]:
features_list=['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

In [7]:
data = pd.read_csv('data/train.csv', index_col = 'id')
data['time'] = data['epoch']
data['epoch'] = pd.to_datetime(data['epoch']).values.astype(float)
data.head()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,time
id,,,,,,,,,,,,,,,
0,1.388534e+18,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,2014-01-01T00:00:00.000
1,1.388537e+18,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,2014-01-01T00:46:43.000
2,1.388540e+18,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768,2014-01-01T01:33:26.001
3,1.388543e+18,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306,2014-01-01T02:20:09.001
4,1.388546e+18,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,2014-01-01T03:06:52.002


In [8]:
test_data = pd.read_csv('data/test.csv', index_col = 'id')
test_data['time'] = test_data['epoch']
test_data['epoch'] = pd.to_datetime(test_data['epoch']).values.astype(float)

In [ ]:
d = data[data.sat_id == 2]
td = test_data[test_data.sat_id == 2]
new = pd.concat([d,td],  axis = 0, sort = False)

In [ ]:
new.tail()

In [ ]:
plt.figure(figsize=(15, 5))

plt.plot(new['epoch'],new['x_sim'])
plt.plot(new['epoch'],new['x'])

In [9]:
result_df = []
alignment_model = LinearAlignment()

satellites_list = test_data['sat_id'].unique()

transf_df = pd.DataFrame([])
for sat_id in tqdm(satellites_list):
    
#     sat_data = utils.get_satellite_data(data, sat_id)
    try:
        sat_data = get_satellite_data(data, sat_id)
        n_first = sat_data.shape[0]
        
        test_sat_data = get_satellite_data(test_data, sat_id)
        
        sat_data = pd.concat([sat_data,test_sat_data], axis = 0, sort = False)
                
        index = sat_data.index
        pred = pd.DataFrame(index = index )
        
        sat_data = remove_time_jumps_fast(sat_data.reset_index(drop=True))
        sat_data.set_index(index)
#         sat_data = get_satellite_data(data, sat_id) # run to check the smape score before any transformations
    except KeyError as e:
        print(f'jump removal failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
    
    
    n_train = 4*len(sat_data) // 10
    
    train_sat_data = sat_data[:n_train]
    
    
    pred['epoch'] = sat_data['epoch'].values
    pred['t'] = sat_data['t'].values
    pred['sat_id'] = sat_id
    
    try:

        for feature_name in features_list:
            alignment_model.fit(t=train_sat_data['epoch'].values,
                                x=-train_sat_data[f'{feature_name}_sim'].values,
                                gt=-train_sat_data[feature_name].values)
            option1 = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
            alignment_model.fit(t=train_sat_data['epoch'].values,
                                x= train_sat_data[f'{feature_name}_sim'].values,
                                gt= train_sat_data[feature_name].values)
            
            option2 = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
            
            if utils.smape(option1[n_train:n_first],sat_data[feature_name].values[n_train:n_first]) < utils.smape(option2[n_train:n_first],sat_data[feature_name].values[n_train:n_first]):
                alignment_model.fit(t=sat_data['epoch'].iloc[:n_first].values,
                                x=-sat_data[f'{feature_name}_sim'].iloc[:n_first].values,
                                gt=-sat_data[feature_name].iloc[:n_first].values)
                    
                pred[f'{feature_name}_sim'] = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
            else:
                alignment_model.fit(t=sat_data['epoch'].iloc[:n_first].values,
                                x=sat_data[f'{feature_name}_sim'].iloc[:n_first].values,
                                gt=sat_data[feature_name].iloc[:n_first].values)
                pred[f'{feature_name}_sim']  = alignment_model.predict(t=sat_data['epoch'].values,
                                                         x=sat_data[f'{feature_name}_sim'].values)
                
            pred.loc[:,feature_name]=sat_data.loc[:, feature_name].values
        
                  
            
    except Exception as e:
        print(f'linear alignment failed for satellite {sat_id}:\t{type(e).__name__} {e}')
        continue
    
    transf_df = transf_df.append(pred)

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]C:\Users\Tatiana\Anaconda3\envs\idao\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:51<00:00,  1.60it/s]


In [10]:
transf_df.head()

,epoch,t,sat_id,x_sim,x,y_sim,y,z_sim,z,Vx_sim,Vx,Vy_sim,Vy,Vz_sim,Vz
id,,,,,,,,,,,,,,,
1819,1.388534e+18,0.000000,1,10405.813755,10390.313089,-2771.180760,-2796.458271,3166.926302,3179.562085,2.508879,2.520477,6.152996,6.149930,-2.826227,-2.827599
1820,1.388536e+18,0.014720,1,11199.459274,11195.606833,5105.200658,5078.653968,-848.597754,-839.076593,-1.132222,-1.126667,5.822436,5.826412,-3.250054,-3.255872
1821,1.388537e+18,0.029441,1,8237.279843,8235.556436,11469.206013,11445.904263,-4684.829256,-4680.514023,-3.228498,-3.230227,4.124297,4.126433,-2.700287,-2.704441
1822,1.388538e+18,0.044161,1,3565.004307,3560.149776,15655.733441,15634.195146,-7655.277835,-7654.177182,-3.960607,-3.964696,2.521337,2.520867,-1.976770,-1.978151
1823,1.388539e+18,0.058882,1,-1571.839073,-1580.476891,18044.683022,18023.318335,-9755.479443,-9755.287599,-4.046854,-4.050865,1.298053,1.296388,-1.347061,-1.346512


In [11]:
transf_df.shape

(599254, 15)

In [ ]:
# # for now, simply append non-transformed values fot the satellites 252
# transf_df = transf_df.append(remove_time_jumps_fast(get_satellite_data(data, 252))[transf_df.columns])
# transf_df = transf_df.append(remove_time_jumps_fast(get_satellite_data(data, 301))[transf_df.columns])
# transf_df.shape

In [ ]:
fail_list = []
satellites_list = transf_df['sat_id'].unique()
res = pd.DataFrame([])
for sat_id in tqdm(satellites_list):
    
    
    
    sat_data = transf_df[transf_df.sat_id == sat_id]
    
    n_train = 4* len(sat_data) // 10
    n_first = len(sat_data[~pd.isna(sat_data.x)])

    pred = pd.DataFrame(index = sat_data.iloc[n_first:].index)
    pred['epoch'] = sat_data['epoch'].iloc[n_first:].values
    pred['t'] = sat_data['t'].iloc[n_first:].values
    pred['sat_id'] = sat_id

    if sat_id in [26]:
        for feature_name in features_list:
            sat_data[f'{feature_name}_sim'].values
    else:
        try:

            for feature_name in features_list:

                q = InterpolatedUnivariateSpline(sat_data['t'][:n_train], sat_data[f'{feature_name}_sim'][:n_train]).roots()
                T = (q[2]-q[0])

                # create a DataFrame for prophet model()
                ts = pd.DataFrame([])
                ts['ds'] = pd.to_datetime(sat_data ['epoch']).values
                ts['y'] = sat_data [feature_name].values-sat_data [f'{feature_name}_sim'].values

                # define the time frame for the first forecast 
                future =ts[n_train:n_first] [['ds']]

                # define the model
                m = Prophet(changepoint_prior_scale = T/20,
                weekly_seasonality = False,
                yearly_seasonality = False).add_seasonality(name='grid', period = T, fourier_order=50)

                #fit the model and make the forecast
                m.fit(ts[:n_train])
                forecast = m.predict(future)

                new_values = sat_data.loc[:,f'{feature_name}_sim'].iloc[n_train:n_first].values+forecast.loc[:, 'yhat'].values

                option1 = sat_data[f'{feature_name}_sim'].values


                option2 = np.concatenate((sat_data[f'{feature_name}_sim'].values[:n_train], new_values), axis=None)


                print('{}[{}]  no correction: {}, correction:{}'.format(sat_id,
                                                                    feature_name,
                                                                    utils.smape(option1[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values),
                                                                    utils.smape(option2[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values)))

                if utils.smape(option1[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values) < utils.smape(option2[n_train:n_first],sat_data[feature_name].iloc[n_train:n_first].values):
                    pred[feature_name] = option1[n_first:]
                    print(sat_id, feature_name, 'does not work')
                    fail_list.append((sat_id, feature_name))
                else:
                    # fit a new model on the whole training range

                    # define the time frame for the first forecast 
                    future =ts[n_first:] [['ds']]

                    # define the model
                    m = Prophet(changepoint_prior_scale = T/20,
                    weekly_seasonality = False,
                    yearly_seasonality = False).add_seasonality(name='grid', period = T, fourier_order=50)

                    #fit the model and make the forecast
                    m.fit(ts[:n_first])
                    forecast = m.predict(future)

                    pred[feature_name]  = sat_data.loc[:,f'{feature_name}_sim'].iloc[n_first:].values+forecast.loc[:, 'yhat'].values





        except Exception as e:
            print(f'phophet failed for satellite {sat_id}:\t{type(e).__name__} {e}')
            continue
        
#     sat_sim_smape = utils.smape(sat_data.loc[n_train:, [f'{f}_sim' for f in features_list]].values,
#                                 sat_data.loc[n_train:, features_list].values)
#     sat_new_smape = utils.smape(pred.loc[n_train:, features_list].values,
#                                 sat_data.loc[n_train:, features_list].values)
#     result_df.append([sat_id, sat_sim_smape, sat_new_smape])
    res = transf_df.append(pred, sort=False)

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]C:\Users\Tatiana\Anaconda3\envs\idao\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


1[x]  no correction: 0.19119642955303978, correction:0.038746070579538786
1[y]  no correction: 0.18267242666654607, correction:0.0015251064080587693
1[z]  no correction: 0.17772439658633013, correction:0.10822315641831583
1[Vx]  no correction: 0.1498960586239248, correction:0.13625637376844116
1[Vy]  no correction: 0.1736203022100542, correction:0.0035356489053515474
1[Vz]  no correction: 0.20008292281196063, correction:0.1050866373640872


  0%|▎                                                                             | 1/300 [05:09<25:41:15, 309.28s/it]

2[x]  no correction: 0.00641881405571727, correction:0.0030207699385655998
2[y]  no correction: 0.010210951558239532, correction:0.006725123878493333
2[z]  no correction: 0.012744072856426519, correction:0.0027442029237939626
2[Vx]  no correction: 0.004361435683741954, correction:0.0021617963233998847
2[Vy]  no correction: 0.0035555157849736342, correction:0.0006616367678082797
2[Vz]  no correction: 0.005076760787618269, correction:0.00195590903393318


  1%|▌                                                                             | 2/300 [06:36<20:05:51, 242.79s/it]

...

522[x]  no correction: 0.12769552196546136, correction:0.06087637935004979
522[y]  no correction: 0.20766854585643565, correction:0.05438831176621626
522[z]  no correction: 0.23997178470313663, correction:0.002336371892652934
522[Vx]  no correction: 0.12349418717283815, correction:0.013027624870851255
522[Vy]  no correction: 0.19361187091272708, correction:0.0038622707323452904
522[Vz]  no correction: 0.23628850853297304, correction:0.18981971165972766


 89%|██████████████████████████████████████████████████████████████████        | 268/300 [19:31:58<5:00:45, 563.93s/it]

523[x]  no correction: 0.506828521768017, correction:0.48700866445337465
523[y]  no correction: 0.5275252746434327, correction:0.4970268037175962
523[z]  no correction: 0.5051122890471778, correction:0.46705459122821363
523[Vx]  no correction: 0.5699025490837644, correction:0.490823723141445
523[Vy]  no correction: 0.5265602671917049, correction:0.5106653606724251
523[Vz]  no correction: 0.5758208360803707, correction:0.5950976391317095
523 Vz does not work


 90%|██████████████████████████████████████████████████████████████████▎       | 269/300 [19:33:28<3:37:51, 421.66s/it]

524[x]  no correction: 0.01586825455145314, correction:0.3906643560417187
524 x does not work
524[y]  no correction: 0.011918289852931148, correction:0.02867115606453627
524 y does not work
524[z]  no correction: 0.0064824966701819454, correction:0.4543350442561822
524 z does not work
